In [22]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
import pandas as pd
import nltk
import re
# from gensim.models import Word2Vec
# import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
from sklearn.metrics import classification_report
from sklearn.preprocessing import scale
import numpy as np
import pandas as pd
import os, sys, getopt, csv, sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
import itertools
from textblob import TextBlob
from keras.models import Sequential
from keras.layers import Dense

In [23]:
data = pd.read_csv(r"C:\Users\konark\Downloads\punjabi_data.csv", encoding='latin-1', engine='python' )
data['Text'] = data['Text'].str.lower()

In [24]:
vocabulary = pd.read_csv(r"C:\Users\konark\Downloads\generalised_spellings.csv")
vocabulary['Words'] = vocabulary['Words'].str.lower()
vocabulary['Generalised Spelling'] = vocabulary['Generalised Spelling'].str.lower()
voc_to_be_used = dict(zip(vocabulary['Words'], vocabulary['Generalised Spelling']))

In [25]:
replacement_dict = {r'(\b){}(\b)'.format(k):r'\1{}\2'.format(v) for k,v in voc_to_be_used.items()}
data['Text'] = data['Text'].replace(replacement_dict, regex=True)

stopwords = ["100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", '10' ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae"]
# stopwords = ["lokk",'abp","video", "aafull", "me", "rhe", "100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", '10' ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae", "bs", "chal", "jagdeep", "captian", "anchor", "ranjeet", "amrinder", "ikk", "ina", "dilli", "mai", "te", "fr","india", "singa", "tn", "dharti", "gurdas", "singer", "sound", "live", "respiratory", "punjab", "punjabian", "jd", "chd", "geet", "ta", "bt", "koe", "nusrat", "fateh", "sudhu", "sidhu", "sukhbir", "interview", "tumhe", "agr", "eh", "uncle", "but", "covid", "dong", "jidaan", "matlab", "tere", "abc", "ik", "for","news", "sanja", "tuc", "tum", "detail", "account", "accounts", "interview", "or", "diya", "add", "addmission", "address", "ade", "admin", "advocate"]
pat = r'\b(?:{})\b'.format('|'.join(stopwords))
data['Text'] = data['Text'].str.replace(pat, '')
data['Text'] = data['Text'].str.replace(r'\s+', ' ')
stopwords = ["lokk","abp","video", "aafull", "me", "rhe", "100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", "10" ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae", "bs", "chal", "jagdeep", "captian", "anchor", "ranjeet", "amrinder", "ikk", "ina", "dilli", "mai", "te", "fr","india", "singa", "tn", "dharti", "gurdas", "singer", "sound", "live", "respiratory", "punjab", "punjabian", "jd", "chd", "geet", "ta", "bt", "koe", "nusrat", "fateh", "sudhu", "sidhu", "sukhbir", "interview", "tumhe", "agr", "eh", "uncle", "but", "covid", "dong", "jidaan", "matlab", "tere", "abc", "ik", "for","news", "sanja", "tuc", "tum", "detail", "account", "accounts", "interview", "or", "diya", "add", "addmission", "address", "ade", "admin", "advocate"]
pat = r'\b(?:{})\b'.format('|'.join(stopwords))
data['Text'] = data['Text'].str.replace(pat, '')
data['Text'] = data['Text'].str.replace(r'\s+', ' ')
stopwords = ["ah","mnu","mai", "100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", '10' ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae"]
pat = r'\b(?:{})\b'.format('|'.join(stopwords))
data['Text'] = data['Text'].str.replace(pat, '')
data['Text'] = data['Text'].str.replace(r'\s+', ' ')
import itertools
# data['Text'] = ''.join(''.join(s)[:2] for _, s in itertools.groupby(data['Text']))
for i in range(len(data)):
#     print (i)
    data['Text'][i] = ''.join(''.join(s)[:2] for _, s in itertools.groupby(data['Text'][i]))

C:\Users\konark\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [26]:
# Preprocessing
def tokens(message):
#     message = unicode(message, 'utf8')
    return TextBlob(message).words

def lemmas(message):
#     message = unicode(message, 'utf8').lower()
    words = TextBlob(message).words
    return [word.lemma for word in words]

In [27]:
# Divide data into Test and Train
msg_train, msg_test, label_train, label_test = train_test_split(data['Text'], data['Sentiment'], test_size=0.15, random_state=42)

In [28]:
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,3))),('tfidf', TfidfTransformer()),('classifier', MultinomialNB())])
# pipeline parameters to automatically explore and tune
params = {
'tfidf__use_idf': (True, False),
'bow__analyzer': (lemmas, tokens),
'tfidf__norm': ('l1', 'l2'),
'classifier__alpha': [1, 1e-1, 1e-2, 1e-3]
}
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
nb_detector = grid.fit(msg_train, label_train)

In [29]:
print (confusion_matrix(label_test, nb_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, nb_detector.predict(msg_test)))

[[239  13  18]
 [ 75  33  37]
 [ 35   9 194]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.68      0.89      0.77       270
           1       0.60      0.23      0.33       145
           2       0.78      0.82      0.80       238

    accuracy                           0.71       653
   macro avg       0.69      0.64      0.63       653
weighted avg       0.70      0.71      0.68       653



In [30]:
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas)),('tfidf', TfidfTransformer()),('classifier', MultinomialNB())])
# pipeline parameters to automatically explore and tune
params = {
'tfidf__use_idf': (True, False),
'bow__analyzer': (lemmas, tokens),
'tfidf__norm': ('l1', 'l2'),
'classifier__alpha': [1, 1e-1, 1e-2, 1e-3]
}
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
nb_detector = grid.fit(msg_train, label_train)

In [31]:
print (confusion_matrix(label_test, nb_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, nb_detector.predict(msg_test)))

[[239  13  18]
 [ 75  33  37]
 [ 35   9 194]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.68      0.89      0.77       270
           1       0.60      0.23      0.33       145
           2       0.78      0.82      0.80       238

    accuracy                           0.71       653
   macro avg       0.69      0.64      0.63       653
weighted avg       0.70      0.71      0.68       653



In [32]:
# Uni-Bi
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,2))),('tfidf', TfidfTransformer()),('classifier', MultinomialNB())])
# pipeline parameters to automatically explore and tune
params = {
'tfidf__use_idf': (True, False),
'bow__analyzer': (lemmas, tokens),
'tfidf__norm': ('l1', 'l2'),
'classifier__alpha': [1, 1e-1, 1e-2, 1e-3]
}
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
nb_detector = grid.fit(msg_train, label_train)

In [33]:
print (confusion_matrix(label_test, nb_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, nb_detector.predict(msg_test)))

[[239  13  18]
 [ 75  33  37]
 [ 35   9 194]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.68      0.89      0.77       270
           1       0.60      0.23      0.33       145
           2       0.78      0.82      0.80       238

    accuracy                           0.71       653
   macro avg       0.69      0.64      0.63       653
weighted avg       0.70      0.71      0.68       653



In [34]:
###############################################SUPPORT VECTOR MACHINES##########################################################
from sklearn.svm import SVC, LinearSVC
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,3))),('tfidf', TfidfTransformer()),('classifier', SVC())])
# pipeline parameters to automatically explore and tune
# params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
params = [
        {'classifier__C': [1, 10, 100, 1000], 'classifier__kernel': ['linear']},
        {'classifier__C': [1, 10, 100, 1000], 'classifier__gamma': [0.001, 0.0001], 'classifier__kernel': ['rbf', 'linear']},
]
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
svc_detector = grid.fit(msg_train, label_train)

In [35]:
print (confusion_matrix(label_test, svc_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, svc_detector.predict(msg_test)))

[[226  22  22]
 [ 59  52  34]
 [ 29  20 189]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.72      0.84      0.77       270
           1       0.55      0.36      0.44       145
           2       0.77      0.79      0.78       238

    accuracy                           0.72       653
   macro avg       0.68      0.66      0.66       653
weighted avg       0.70      0.72      0.70       653



In [36]:
# Uni-bi
from sklearn.svm import SVC, LinearSVC
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,2))),('tfidf', TfidfTransformer()),('classifier', SVC())])
# pipeline parameters to automatically explore and tune
# params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
params = [
        {'classifier__C': [1, 10, 100, 1000], 'classifier__kernel': ['linear']},
        {'classifier__C': [1, 10, 100, 1000], 'classifier__gamma': [0.001, 0.0001], 'classifier__kernel': ['rbf', 'linear']},
]
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
svc_detector = grid.fit(msg_train, label_train)

print (confusion_matrix(label_test, svc_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, svc_detector.predict(msg_test)))

[[226  22  22]
 [ 59  52  34]
 [ 29  20 189]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.72      0.84      0.77       270
           1       0.55      0.36      0.44       145
           2       0.77      0.79      0.78       238

    accuracy                           0.72       653
   macro avg       0.68      0.66      0.66       653
weighted avg       0.70      0.72      0.70       653



In [37]:
# Uni
from sklearn.svm import SVC, LinearSVC
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas)),('tfidf', TfidfTransformer()),('classifier', SVC())])
# pipeline parameters to automatically explore and tune
# params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
params = [
        {'classifier__C': [1, 10, 100, 1000], 'classifier__kernel': ['linear']},
        {'classifier__C': [1, 10, 100, 1000], 'classifier__gamma': [0.001, 0.0001], 'classifier__kernel': ['rbf', 'linear']},
]
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
svc_detector = grid.fit(msg_train, label_train)

print (confusion_matrix(label_test, svc_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, svc_detector.predict(msg_test)))

[[226  22  22]
 [ 59  52  34]
 [ 29  20 189]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.72      0.84      0.77       270
           1       0.55      0.36      0.44       145
           2       0.77      0.79      0.78       238

    accuracy                           0.72       653
   macro avg       0.68      0.66      0.66       653
weighted avg       0.70      0.72      0.70       653



In [38]:
#############SGC CLassifier##################################
from sklearn.linear_model import SGDClassifier
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,3))),('tfidf', TfidfTransformer()),('classifier', SGDClassifier(max_iter=1000))])
# pipeline parameters to automatically explore and tune
# params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
params = {
'tfidf__use_idf': (True, False),
'bow__analyzer': (lemmas, tokens),
'tfidf__norm': ('l1', 'l2'),
'classifier__loss': ["hinge", "log", "squared_hinge", "modified_huber"],
"classifier__alpha" : [0.0001, 0.001, 0.01, 0.1],
'classifier__penalty' : ["l2", "l1", "none"]
}

skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
sgd_detector = grid.fit(msg_train, label_train)

In [39]:
print (confusion_matrix(label_test, sgd_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, sgd_detector.predict(msg_test)))

[[228  21  21]
 [ 58  47  40]
 [ 29  17 192]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.72      0.84      0.78       270
           1       0.55      0.32      0.41       145
           2       0.76      0.81      0.78       238

    accuracy                           0.72       653
   macro avg       0.68      0.66      0.66       653
weighted avg       0.70      0.72      0.70       653



In [40]:
#Uni-Bi
#############SGC CLassifier##################################
from sklearn.linear_model import SGDClassifier
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,2))),('tfidf', TfidfTransformer()),('classifier', SGDClassifier(max_iter=1000))])
# pipeline parameters to automatically explore and tune
# params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
params = {
'tfidf__use_idf': (True, False),
'bow__analyzer': (lemmas, tokens),
'tfidf__norm': ('l1', 'l2'),
'classifier__loss': ["hinge", "log", "squared_hinge", "modified_huber"],
"classifier__alpha" : [0.0001, 0.001, 0.01, 0.1],
'classifier__penalty' : ["l2", "l1", "none"]
}

skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
sgd_detector = grid.fit(msg_train, label_train)

print (confusion_matrix(label_test, sgd_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, sgd_detector.predict(msg_test)))

[[227  21  22]
 [ 50  52  43]
 [ 24  20 194]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.75      0.84      0.80       270
           1       0.56      0.36      0.44       145
           2       0.75      0.82      0.78       238

    accuracy                           0.72       653
   macro avg       0.69      0.67      0.67       653
weighted avg       0.71      0.72      0.71       653



In [41]:
#Uni
#############SGC CLassifier##################################
from sklearn.linear_model import SGDClassifier
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas)),('tfidf', TfidfTransformer()),('classifier', SGDClassifier(max_iter=1000))])
# pipeline parameters to automatically explore and tune
# params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
params = {
'tfidf__use_idf': (True, False),
'bow__analyzer': (lemmas, tokens),
'tfidf__norm': ('l1', 'l2'),
'classifier__loss': ["hinge", "log", "squared_hinge", "modified_huber"],
"classifier__alpha" : [0.0001, 0.001, 0.01, 0.1],
'classifier__penalty' : ["l2", "l1", "none"]
}

skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
sgd_detector = grid.fit(msg_train, label_train)

print (confusion_matrix(label_test, sgd_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, sgd_detector.predict(msg_test)))

[[226  21  23]
 [ 56  48  41]
 [ 29  16 193]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.73      0.84      0.78       270
           1       0.56      0.33      0.42       145
           2       0.75      0.81      0.78       238

    accuracy                           0.72       653
   macro avg       0.68      0.66      0.66       653
weighted avg       0.70      0.72      0.70       653



In [43]:
##########################LSTM IMPLEMENTATION##############################
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
#LSTM Implementation
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 14000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data['Text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9576 unique tokens.


In [44]:
X = tokenizer.texts_to_sequences(data['Text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (4349, 250)


In [45]:
data.head()

,Text,Sentiment
0,hindu rosh nae hunda,1
1,moot peeniya koda lgda veer walay salaam kalam,2
2,"siraa karta bro, att",2
3,moot peene moot he pee bus,0
4,sahi song,2


In [47]:
Y = pd.get_dummies(data['Sentiment']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (4349, 3)


In [48]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(3914, 250) (3914, 3)
(435, 250) (435, 3)


In [49]:
# variables_for_classification=3 #change it as per your number of categories
model = Sequential()
# model.add(Dense(variables_for_classification, activation='softmax'))
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(13, activation='softmax'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 100)          1400000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 250, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 13)                1313      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 42        
Total params: 1,481,755
Trainable params: 1,481,755
Non-trainable params: 0
_________________________________________________________________
None


In [50]:
epochs = 100
batch_size = 64
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Instructions for updating:
Use tf.cast instead.
Train on 3522 samples, validate on 392 samples
Epoch 1/100
3522/3522 [==============================] - 35s 10ms/step - loss: 1.0706 - accuracy: 0.3958 - val_loss: 1.0536 - val_accuracy: 0.4362
Epoch 2/100
3522/3522 [==============================] - 34s 10ms/step - loss: 1.0500 - accuracy: 0.4514 - val_loss: 1.0015 - val_accuracy: 0.6173
Epoch 3/100
3522/3522 [==============================] - 33s 9ms/step - loss: 0.9479 - accuracy: 0.6715 - val_loss: 0.9324 - val_accuracy: 0.6607
Epoch 4/100
3522/3522 [==============================] - 36s 10ms/step - loss: 0.8696 - accuracy: 0.7286 - val_loss: 0.8974 - val_accuracy: 0.6633
Epoch 5/100
3522/3522 [==============================] - 36s 10ms/step - loss: 0.8102 - accuracy: 0.7530 - val_loss: 0.8732 - val_accuracy: 0.6760
Epoch 6/100
3522/3522 [==============================] - 35s 10ms/step - loss: 0.7660 - accuracy: 0.7621 - val_loss: 0.8719 - val_accuracy: 0.6607
Epoch 7/100
3522/3522 [=

In [52]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f} %'.format(accr[0],accr[1]*100))

435/435 [==============================] - 1s 3ms/step
Test set
  Loss: 1.607
  Accuracy: 61.379


In [56]:
y_pred = model.predict(X_test, verbose=0)

In [57]:
import sklearn.metrics as metrics
matrix = metrics.confusion_matrix(Y_test.argmax(axis=1), y_pred.argmax(axis=1))

In [60]:
print (confusion_matrix(Y_test.argmax(axis=1), y_pred.argmax(axis=1)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(Y_test.argmax(axis=1), y_pred.argmax(axis=1)))

[[125  44  15]
 [ 21  44  34]
 [ 23  31  98]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.74      0.68      0.71       184
           1       0.37      0.44      0.40        99
           2       0.67      0.64      0.66       152

    accuracy                           0.61       435
   macro avg       0.59      0.59      0.59       435
weighted avg       0.63      0.61      0.62       435



In [62]:
from xgboost import XGBClassifier
import xgboost
######XGBoost Implementation####################
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,3))),('tfidf', TfidfTransformer()),('classifier', XGBClassifier())])

params = {'tfidf__use_idf': (True, False),'bow__analyzer': (lemmas, tokens),'tfidf__norm': ('l1', 'l2'),'classifier__min_child_weight': [5, 10], 'classifier__gamma': [2, 5], 'classifier__subsample': [0.6], 'classifier__max_depth': [5,10]}

skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
xgb_detector = grid.fit(msg_train, label_train)

import numpy as np
# temsg_testdiction_list = np.array(xgb_detector.predict(msg_test))
print (confusion_matrix(label_test, xgb_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, xgb_detector.predict(msg_test)))

[[209  27  34]
 [ 56  46  43]
 [ 31  19 188]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.71      0.77      0.74       270
           1       0.50      0.32      0.39       145
           2       0.71      0.79      0.75       238

    accuracy                           0.68       653
   macro avg       0.64      0.63      0.62       653
weighted avg       0.66      0.68      0.66       653



In [63]:
###################Uni-Bi#######################
from xgboost import XGBClassifier
import xgboost
######XGBoost Implementation####################
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,2))),('tfidf', TfidfTransformer()),('classifier', XGBClassifier())])

params = {'tfidf__use_idf': (True, False),'bow__analyzer': (lemmas, tokens),'tfidf__norm': ('l1', 'l2'),'classifier__min_child_weight': [5, 10], 'classifier__gamma': [2, 5], 'classifier__subsample': [0.6], 'classifier__max_depth': [5,10]}

skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
xgb_detector = grid.fit(msg_train, label_train)

import numpy as np
# temsg_testdiction_list = np.array(xgb_detector.predict(msg_test))
print (confusion_matrix(label_test, xgb_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, xgb_detector.predict(msg_test)))

[[209  27  34]
 [ 56  46  43]
 [ 31  19 188]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.71      0.77      0.74       270
           1       0.50      0.32      0.39       145
           2       0.71      0.79      0.75       238

    accuracy                           0.68       653
   macro avg       0.64      0.63      0.62       653
weighted avg       0.66      0.68      0.66       653



In [64]:
from xgboost import XGBClassifier
import xgboost
######XGBoost Implementation####################
#############Uni######
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas)),('tfidf', TfidfTransformer()),('classifier', XGBClassifier())])

params = {'tfidf__use_idf': (True, False),'bow__analyzer': (lemmas, tokens),'tfidf__norm': ('l1', 'l2'),'classifier__min_child_weight': [5, 10], 'classifier__gamma': [2, 5], 'classifier__subsample': [0.6], 'classifier__max_depth': [5,10]}

skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
xgb_detector = grid.fit(msg_train, label_train)

import numpy as np
# temsg_testdiction_list = np.array(xgb_detector.predict(msg_test))
print (confusion_matrix(label_test, xgb_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, xgb_detector.predict(msg_test)))

[[209  27  34]
 [ 56  46  43]
 [ 31  19 188]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.71      0.77      0.74       270
           1       0.50      0.32      0.39       145
           2       0.71      0.79      0.75       238

    accuracy                           0.68       653
   macro avg       0.64      0.63      0.62       653
weighted avg       0.66      0.68      0.66       653



In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range = (1,3))
data_text = vectorizer.fit_transform(data['Text'])
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
msg_train, msg_test, label_train, label_test = train_test_split(data_text, data['Sentiment'], test_size=0.2, random_state=42)
###SVM##########################
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier

# pipe1 = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,3))),('classifier', MultinomialNB())])
# pipe2 = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,3))),('classifier', SVC())])
# pipe3 = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,3))),('classifier', SGDClassifier(max_iter=1000))])

mnb = MultinomialNB()
svc = SVC(probability=True)
sgd = SGDClassifier(max_iter=1000)
lr = LogisticRegression(multi_class='multinomial', solver='newton-cg')
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
gbm = GradientBoostingClassifier()
voting_clf = VotingClassifier(estimators=[('SVC', svc),
                                          ('MNB', mnb),
                                          ('LR', lr),
                                          ('SGD', sgd)],
#                                           ('GBM', gbm)],
                                          voting='soft')

params = {'SVC__C': [1, 10, 100], 'SVC__gamma': [0.001], 'SVC__kernel': ['rbf', 'linear'],
          'MNB__alpha': [1, 1e-1, 1e-2],
          'LR__penalty' : ['l2'],'LR__C' : np.logspace(-4, 4, 4),
          'SGD__loss': ["log", "modified_huber"],"SGD__alpha" : [0.001, 0.01],
          'SGD__penalty' : ["l2", "l1"]}
#           'GBM__loss':["deviance"], 'GBM__max_features': ['auto', 'sqrt', 'log2'], 'GBM__max_depth':[3,5,8], 'GBM__max_features':["log2","sqrt"]}                                          
skf = StratifiedKFold(n_splits=3)
skf.get_n_splits(msg_train, label_train)

grid = GridSearchCV(
    voting_clf,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)

ensembling_detector = grid.fit(msg_train, label_train)
print (confusion_matrix(label_test, ensembling_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, ensembling_detector.predict(msg_test)))

[[309  27  21]
 [ 75  77  38]
 [ 40  29 254]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.73      0.87      0.79       357
           1       0.58      0.41      0.48       190
           2       0.81      0.79      0.80       323

    accuracy                           0.74       870
   macro avg       0.71      0.69      0.69       870
weighted avg       0.73      0.74      0.73       870



In [70]:

import joblib
filename = r'C:\Users\konark\Downloads\finalized_model_v4.sav'
joblib.dump(ensembling_detector, filename)                                          

['C:\\Users\\konark\\Downloads\\finalized_model_v4.sav']

In [145]:
data = pd.read_csv(r"C:\Users\konark\Downloads\punjabi_data_v2.csv", encoding='latin-1', engine='python' )
data['Text'] = data['Text'].str.lower()

vocabulary = pd.read_csv(r"C:\Users\konark\Downloads\generalised_spellings.csv")
vocabulary['Words'] = vocabulary['Words'].str.lower()
vocabulary['Generalised Spelling'] = vocabulary['Generalised Spelling'].str.lower()
voc_to_be_used = dict(zip(vocabulary['Words'], vocabulary['Generalised Spelling']))

replacement_dict = {r'(\b){}(\b)'.format(k):r'\1{}\2'.format(v) for k,v in voc_to_be_used.items()}
data['Text'] = data['Text'].replace(replacement_dict, regex=True)

stopwords = ["100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", '10' ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae"]
# stopwords = ["lokk",'abp","video", "aafull", "me", "rhe", "100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", '10' ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae", "bs", "chal", "jagdeep", "captian", "anchor", "ranjeet", "amrinder", "ikk", "ina", "dilli", "mai", "te", "fr","india", "singa", "tn", "dharti", "gurdas", "singer", "sound", "live", "respiratory", "punjab", "punjabian", "jd", "chd", "geet", "ta", "bt", "koe", "nusrat", "fateh", "sudhu", "sidhu", "sukhbir", "interview", "tumhe", "agr", "eh", "uncle", "but", "covid", "dong", "jidaan", "matlab", "tere", "abc", "ik", "for","news", "sanja", "tuc", "tum", "detail", "account", "accounts", "interview", "or", "diya", "add", "addmission", "address", "ade", "admin", "advocate"]
pat = r'\b(?:{})\b'.format('|'.join(stopwords))
data['Text'] = data['Text'].str.replace(pat, '')
data['Text'] = data['Text'].str.replace(r'\s+', ' ')
stopwords = ["lokk","abp","video", "aafull", "me", "rhe", "100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", "10" ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae", "bs", "chal", "jagdeep", "captian", "anchor", "ranjeet", "amrinder", "ikk", "ina", "dilli", "mai", "te", "fr","india", "singa", "tn", "dharti", "gurdas", "singer", "sound", "live", "respiratory", "punjab", "punjabian", "jd", "chd", "geet", "ta", "bt", "koe", "nusrat", "fateh", "sudhu", "sidhu", "sukhbir", "interview", "tumhe", "agr", "eh", "uncle", "but", "covid", "dong", "jidaan", "matlab", "tere", "abc", "ik", "for","news", "sanja", "tuc", "tum", "detail", "account", "accounts", "interview", "or", "diya", "add", "addmission", "address", "ade", "admin", "advocate"]
pat = r'\b(?:{})\b'.format('|'.join(stopwords))
data['Text'] = data['Text'].str.replace(pat, '')
data['Text'] = data['Text'].str.replace(r'\s+', ' ')
stopwords = ["kasoor", "ssp", "prateek","tu", "wabba", "pyn", "ah","mnu","mai", "100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", '10' ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae"]
pat = r'\b(?:{})\b'.format('|'.join(stopwords))
data['Text'] = data['Text'].str.replace(pat, '')
data['Text'] = data['Text'].str.replace(r'\s+', ' ')
import itertools
for i in range(len(data)):
    data['Text'][i] = ''.join(''.join(s)[:2] for _, s in itertools.groupby(data['Text'][i]))

C:\Users\konark\anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [146]:
########LSTM + WORD2VEC+TF-IDF##################

def cleaning_text(original_series):
    original_series = original_series.str.replace('(ap)', '')
    converting_nt = original_series.str.replace(r'n\'t',' not')
    
    removing_trailing_spaces_lowering = converting_nt.str.strip().str.lower()
    
    removing_special_characters = removing_trailing_spaces_lowering.str.replace(r'[!.:,#-$\\\'0-9]','')
    removing_bracket_starting = removing_special_characters.str.replace(r'\(','')
    removing_bracket_ending = removing_bracket_starting.str.replace(r'\)','')
    splitting_into_words = removing_bracket_ending.str.split()
    words_greater_than_zero = splitting_into_words[splitting_into_words.map(lambda d: len(d)) > 0]
    return (words_greater_than_zero)

clean_sentences = cleaning_text(data['Text'])

In [147]:
X_train, X_test, y_train, y_test = train_test_split(data['Text'], data.Sentiment, test_size=.2)

In [148]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
LabeledSentence = gensim.models.doc2vec.LabeledSentence

def labelizeStories(stories, label_type):
    labelized = []
    for i,v in enumerate(stories):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

X_train = labelizeStories(X_train, 'TRAIN')
X_test = labelizeStories(X_test, 'TEST')

C:\Users\konark\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  if __name__ == '__main__':


In [149]:
X_train[0]

LabeledSentence(words=' gapa wah wah jhooth', tags=['TRAIN_0'])

In [150]:
from gensim.models import Word2Vec

model_w2v = Word2Vec(list(clean_sentences), min_count=2,sg=1,window=8)

In [151]:
print (model_w2v)

Word2Vec(vocab=4496, size=100, alpha=0.025)


In [152]:
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
output_notebook()
plot_tfidf = bp.figure(plot_width=700, plot_height=600, title="A map of 4210 word vectors",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

Loading BokehJS ...

In [153]:
word_vectors = [model_w2v[w] for w in model_w2v.wv.vocab.keys()]

C:\Users\konark\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [154]:
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_w2v = tsne_model.fit_transform(word_vectors)

# putting everything in a dataframe
tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
tsne_df['words'] = model_w2v.wv.vocab.keys()

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 4496 samples in 0.146s...
[t-SNE] Computed neighbors for 4496 samples in 1.538s...
[t-SNE] Computed conditional probabilities for sample 1000 / 4496
[t-SNE] Computed conditional probabilities for sample 2000 / 4496
[t-SNE] Computed conditional probabilities for sample 3000 / 4496
[t-SNE] Computed conditional probabilities for sample 4000 / 4496
[t-SNE] Computed conditional probabilities for sample 4496 / 4496
[t-SNE] Mean sigma: 0.013037
[t-SNE] KL divergence after 250 iterations with early exaggeration: 64.225677
[t-SNE] KL divergence after 1000 iterations: 1.044657


In [155]:
plot_tfidf.scatter(x='x', y='y', source=tsne_df)
hover = plot_tfidf.select(dict(type=HoverTool))
hover.tooltips={"word": "@words"}
show(plot_tfidf)

In [156]:
print ('building tf-idf matrix ...')
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=2)
matrix = vectorizer.fit_transform([x.words for x in X_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

building tf-idf matrix ...
vocab size : 48


In [157]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
#     count = 0.
    count = 1
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size)) * tfidf[word]
#             count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [158]:
train_vecs_w2v = np.concatenate([buildWordVector(z, 100) for z in map(lambda x: x.words, X_train)])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, 100) for z in map(lambda x: x.words, X_test)])
test_vecs_w2v = scale(test_vecs_w2v)

C:\Users\konark\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
C:\Users\konark\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [159]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=100))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [160]:
model.fit(train_vecs_w2v, y_train, epochs=100, batch_size=32, verbose=2)

Epoch 1/100
 - 1s - loss: 0.1781 - accuracy: 0.2234
Epoch 2/100
 - 0s - loss: 0.0955 - accuracy: 0.2219
Epoch 3/100
 - 0s - loss: 0.0325 - accuracy: 0.2268
Epoch 4/100
 - 0s - loss: -5.8128e-02 - accuracy: 0.2279
Epoch 5/100
 - 0s - loss: -1.2793e-01 - accuracy: 0.2258
Epoch 6/100
 - 0s - loss: -1.2912e-01 - accuracy: 0.2289
Epoch 7/100
 - 0s - loss: -1.7227e-01 - accuracy: 0.2255
Epoch 8/100
 - 0s - loss: -1.2591e-01 - accuracy: 0.2250
Epoch 9/100
 - 0s - loss: -1.3836e-01 - accuracy: 0.2177
Epoch 10/100
 - 0s - loss: -1.7525e-01 - accuracy: 0.2195
Epoch 11/100
 - 0s - loss: -1.6281e-01 - accuracy: 0.2201
Epoch 12/100
 - 0s - loss: -1.7555e-01 - accuracy: 0.2206
Epoch 13/100
 - 0s - loss: -1.9448e-01 - accuracy: 0.2247
Epoch 14/100
 - 0s - loss: -2.1583e-01 - accuracy: 0.2245
Epoch 15/100
 - 0s - loss: -2.0660e-01 - accuracy: 0.2284
Epoch 16/100
 - 0s - loss: -2.7929e-01 - accuracy: 0.2333
Epoch 17/100
 - 0s - loss: -3.0401e-01 - accuracy: 0.2292
Epoch 18/100
 - 0s - loss: -3.5774e-01

In [161]:
score = model.evaluate(test_vecs_w2v, y_test, batch_size=128, verbose=2)

In [162]:
print ('Accuracy', score[1]*100,'%')

Accuracy 25.33748745918274 %


In [1]:
###############################BI-DIRECTIONAL LSTM##########################################
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers


from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
import tensorflow as tf
import os
import time
import gc
import re
import glob

Using TensorFlow backend.


In [4]:
# Define some Global Variables
max_features = 100000 # Maximum Number of words we want to include in our dictionary
maxlen = 72 # No of words in question we want to create a sequence with
embed_size = 300# Size of word to vec embedding we are using

In [3]:
data = pd.read_csv(r"C:\Users\User\Eng-Pun_report\punjabi_data_v2.csv", encoding='latin-1', engine='python' )
data['Text'] = data['Text'].str.lower()

vocabulary = pd.read_csv(r"C:\Users\User\Eng-Pun_report\generalised_spellings.csv")
vocabulary['Words'] = vocabulary['Words'].str.lower()
vocabulary['Generalised Spelling'] = vocabulary['Generalised Spelling'].str.lower()
voc_to_be_used = dict(zip(vocabulary['Words'], vocabulary['Generalised Spelling']))

replacement_dict = {r'(\b){}(\b)'.format(k):r'\1{}\2'.format(v) for k,v in voc_to_be_used.items()}
data['Text'] = data['Text'].replace(replacement_dict, regex=True)

stopwords = ["100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", '10' ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae"]
# stopwords = ["lokk",'abp","video", "aafull", "me", "rhe", "100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", '10' ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae", "bs", "chal", "jagdeep", "captian", "anchor", "ranjeet", "amrinder", "ikk", "ina", "dilli", "mai", "te", "fr","india", "singa", "tn", "dharti", "gurdas", "singer", "sound", "live", "respiratory", "punjab", "punjabian", "jd", "chd", "geet", "ta", "bt", "koe", "nusrat", "fateh", "sudhu", "sidhu", "sukhbir", "interview", "tumhe", "agr", "eh", "uncle", "but", "covid", "dong", "jidaan", "matlab", "tere", "abc", "ik", "for","news", "sanja", "tuc", "tum", "detail", "account", "accounts", "interview", "or", "diya", "add", "addmission", "address", "ade", "admin", "advocate"]
pat = r'\b(?:{})\b'.format('|'.join(stopwords))
data['Text'] = data['Text'].str.replace(pat, '')
data['Text'] = data['Text'].str.replace(r'\s+', ' ')
stopwords = ["lokk","abp","video", "aafull", "me", "rhe", "100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", "10" ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae", "bs", "chal", "jagdeep", "captian", "anchor", "ranjeet", "amrinder", "ikk", "ina", "dilli", "mai", "te", "fr","india", "singa", "tn", "dharti", "gurdas", "singer", "sound", "live", "respiratory", "punjab", "punjabian", "jd", "chd", "geet", "ta", "bt", "koe", "nusrat", "fateh", "sudhu", "sidhu", "sukhbir", "interview", "tumhe", "agr", "eh", "uncle", "but", "covid", "dong", "jidaan", "matlab", "tere", "abc", "ik", "for","news", "sanja", "tuc", "tum", "detail", "account", "accounts", "interview", "or", "diya", "add", "addmission", "address", "ade", "admin", "advocate"]
pat = r'\b(?:{})\b'.format('|'.join(stopwords))
data['Text'] = data['Text'].str.replace(pat, '')
data['Text'] = data['Text'].str.replace(r'\s+', ' ')
stopwords = ["k","c","kasoor", "ssp", "prateek","tu", "wabba", "pyn", "ah","mnu","mai", "100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", '10' ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae"]
pat = r'\b(?:{})\b'.format('|'.join(stopwords))
data['Text'] = data['Text'].str.replace(pat, '')
data['Text'] = data['Text'].str.replace(r'\s+', ' ')
import itertools
for i in range(len(data)):
    data['Text'][i] = ''.join(''.join(s)[:2] for _, s in itertools.groupby(data['Text'][i]))

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
tokenizer = Tokenizer(num_words=2000, split=' ')

tokenizer.fit_on_texts(data['Text'])
X = tokenizer.texts_to_sequences(data['Text'])
X = pad_sequences(X)
# Y = data['Sentiment']

Y = pd.get_dummies(data['Sentiment']).values
print('Shape of label tensor:', Y.shape)

# We can then create our train and test sets:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15, random_state = 42,stratify=Y)

# X_train, X_test, y_train, y_test = train_test_split(data['Text'], data.Sentiment, test_size=.15)

Shape of label tensor: (4812, 3)


In [8]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(4090, 160) (4090, 3)
(722, 160) (722, 3)


In [9]:
max_features = 20000
# cut texts after this number of words
# (among top max_features most common words)
maxlen = 100
batch_size = 32

In [10]:
from keras.preprocessing import sequence
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
y_train = np.array(Y_train)
y_test = np.array(Y_test)

Pad sequences (samples x time)
x_train shape: (4090, 100)
x_test shape: (722, 100)


In [11]:
# model = Sequential()
# model.add(Embedding(11000, embed_dim = 128, input_length = X.shape[1], dropout=0.2))
# model.add( Bidirectional( LSTM(lstm_out = 196, dropout_U = 0.2, dropout_W = 0.2)))
# model.add( Dense(3, activation = 'softmax'))

# model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(3, activation='sigmoid'))

In [12]:
#  try using different optimizers and different optimizer configs
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [13]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=100,
#           validation_data=[x_test, y_test]
         )

Train...


C:\Users\User\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
4090/4090 [==============================] - 25s 6ms/step - loss: 0.5696 - accuracy: 0.7125
Epoch 2/100
4090/4090 [==============================] - 20s 5ms/step - loss: 0.3912 - accuracy: 0.8227
Epoch 3/100
4090/4090 [==============================] - 20s 5ms/step - loss: 0.3093 - accuracy: 0.8645
Epoch 4/100
4090/4090 [==============================] - 20s 5ms/step - loss: 0.2508 - accuracy: 0.8963
Epoch 5/100
4090/4090 [==============================] - 20s 5ms/step - loss: 0.2078 - accuracy: 0.9164
Epoch 6/100
4090/4090 [==============================] - 20s 5ms/step - loss: 0.1796 - accuracy: 0.9331
Epoch 7/100
4090/4090 [==============================] - 20s 5ms/step - loss: 0.1576 - accuracy: 0.9399
Epoch 8/100
4090/4090 [==============================] - 20s 5ms/step - loss: 0.1404 - accuracy: 0.9488
Epoch 9/100
4090/4090 [==============================] - 20s 5ms/step - loss: 0.1155 - accuracy: 0.9574
Epoch 10/100
4090/4090 [==============================] - 20s 5m

In [14]:
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
accr = model.evaluate(x_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f} %'.format(accr[0],accr[1]*100))

722/722 [==============================] - 1s 840us/step
Test set
  Loss: 2.093
  Accuracy: 78.163 %


In [15]:
y_pred = model.predict(x_test, verbose=0)

In [16]:
np.round(y_pred[0],0).astype(int)

array([0, 0, 1])

In [17]:
y_test[0]

array([0, 0, 1], dtype=uint8)

In [18]:
y_pred_max = [np.round(p,0).astype(int) for p in y_pred]

In [19]:
y_pred_max[0]

array([0, 0, 1])

In [27]:
print (":: Classification Report")
print ("")
print (classification_report(y_test, y_pred_max))

:: Classification Report

              precision    recall  f1-score   support

           0       0.73      0.71      0.72       303
           1       0.44      0.42      0.43       158
           2       0.75      0.75      0.75       261

   micro avg       0.68      0.66      0.67       722
   macro avg       0.64      0.63      0.63       722
weighted avg       0.68      0.66      0.67       722
 samples avg       0.66      0.66      0.66       722



In [31]:
import os

os.system('jupyter nbconvert --to html sentiment_analysis_all_algos_results.ipynb')

0